# 동물 전문가 시스템 구현하기

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners)에서 제공하는 예제입니다.

이 예제에서는 몇 가지 신체적 특징을 기반으로 동물을 판별하는 간단한 지식 기반 시스템을 구현합니다. 이 시스템은 다음과 같은 AND-OR 트리로 표현될 수 있습니다 (이 트리는 전체 트리의 일부이며, 규칙을 더 추가하는 것도 간단합니다):

![](../../../../lessons/2-Symbolic/images/AND-OR-Tree.png)


## 우리의 역추론 기반 전문가 시스템 셸

생산 규칙에 기반한 지식 표현을 위한 간단한 언어를 정의해 봅시다. 우리는 규칙을 정의하기 위해 Python 클래스들을 키워드로 사용할 것입니다. 기본적으로 세 가지 유형의 클래스가 있습니다:
* `Ask`는 사용자에게 물어봐야 할 질문을 나타냅니다. 이 클래스는 가능한 답변의 집합을 포함합니다.
* `If`는 규칙을 나타내며, 규칙의 내용을 저장하기 위한 단순한 문법적 설탕(syntactic sugar)입니다.
* `AND`/`OR`는 트리의 AND/OR 분기를 나타내는 클래스입니다. 이 클래스들은 내부에 인수 목록을 저장하는 역할만 합니다. 코드를 단순화하기 위해 모든 기능은 부모 클래스인 `Content`에 정의됩니다.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

우리 시스템에서 작업 기억은 **속성-값 쌍**으로 된 **사실** 목록을 포함합니다. 지식 기반은 작업 기억에 삽입해야 할 새로운 사실(행동)을 조건에 매핑하는 하나의 큰 사전으로 정의될 수 있으며, 조건은 AND-OR 표현식으로 표현됩니다. 또한 일부 사실은 `Ask`될 수 있습니다.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

역추론을 수행하기 위해 `Knowledgebase` 클래스를 정의합니다. 이 클래스는 다음을 포함합니다:
* 작동 중인 `memory` - 속성을 값에 매핑하는 딕셔너리
* 위에서 정의된 형식의 Knowledgebase `rules`

주요 메서드 두 가지는 다음과 같습니다:
* `get` - 속성의 값을 얻고 필요하면 추론을 수행합니다. 예를 들어, `get('color')`는 색상 슬롯의 값을 가져옵니다(필요하면 질문을 하고, 이후 사용을 위해 작업 메모리에 값을 저장합니다). 만약 `get('color:blue')`를 요청하면 색상을 묻고, 색상에 따라 `y`/`n` 값을 반환합니다.
* `eval` - 실제 추론을 수행하며, AND/OR 트리를 탐색하고, 하위 목표를 평가하는 등의 작업을 합니다.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

이제 우리의 동물 지식 기반을 정의하고 상담을 수행해 봅시다. 이 호출은 당신에게 질문을 할 것입니다. 예/아니오 질문에는 `y`/`n`을 입력하여 답변할 수 있으며, 더 긴 다중 선택 질문에는 숫자(0..N)를 지정하여 답변할 수 있습니다.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow를 사용한 전방 추론

다음 예제에서는 지식 표현을 위한 라이브러리 중 하나인 [PyKnow](https://github.com/buguroo/pyknow/)를 사용하여 전방 추론을 구현해 보겠습니다. **PyKnow**는 Python에서 전방 추론 시스템을 생성하기 위한 라이브러리로, 고전적인 시스템 [CLIPS](http://www.clipsrules.net/index.html)와 유사하게 설계되었습니다.

물론 우리가 직접 전방 연쇄를 구현할 수도 있지만, 단순한 구현은 보통 효율적이지 않습니다. 더 효과적인 규칙 매칭을 위해 특별한 알고리즘인 [Rete](https://en.wikipedia.org/wiki/Rete_algorithm)가 사용됩니다.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

우리 시스템을 `KnowledgeEngine`을 서브클래스로 하는 클래스으로 정의할 것입니다. 각 규칙은 `@Rule` 주석으로 정의된 별도의 함수로 구성되며, 규칙이 실행되어야 할 시점을 지정합니다. 규칙 내부에서는 `declare` 함수를 사용하여 새로운 사실을 추가할 수 있으며, 이러한 사실을 추가하면 전방 추론 엔진에 의해 더 많은 규칙이 호출됩니다.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

지식 기반을 정의한 후, 몇 가지 초기 사실로 작업 메모리를 채우고 `run()` 메서드를 호출하여 추론을 수행합니다. 결과적으로 새로운 추론된 사실들이 작업 메모리에 추가되며, 초기 사실을 올바르게 설정한 경우 동물에 대한 최종 사실도 포함됩니다.


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
